# 0. Import Dependencies

In [9]:
import os
import cv2
import numpy as np

import pandas as pd

import joblib

from sklearn.svm import LinearSVC
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split


# 1. Prepare Samples

In [5]:
positive_samples_paths = []
negative_samples_paths = []

positive_samples_df = pd.read_csv("../../datasets/faces/faces.csv")
# # Remove images with more than one face
# positive_samples_df = positive_samples_df.groupby('image_name').filter(lambda x: len(x) == 1).reset_index(drop=True)
# # Shuffle the data frame
# positive_samples_df = positive_samples_df.sample(frac=1).reset_index(drop=True)

negative_samples_dirs = [os.path.join("../../datasets/faces/natural_images/", folder) for folder in os.listdir("../../datasets/faces/natural_images/") if folder != "person"]

for dir in negative_samples_dirs: 
    for image in os.listdir(dir):
        negative_samples_paths.append(os.path.join(dir, image))

for image in os.listdir("../../datasets/faces/natural_images/person/"):
    positive_samples_paths.append(os.path.join("../../datasets/faces/natural_images/person/", image))
# for image in os.listdir("../../datasets/faces/images"):
#     positive_samples_paths.append(os.path.join(dir, image))


# 2. Feature Extraction

In [6]:
feature_vector = []
label_vector = []

FEATURE_WINDOW_SIZE = (64, 64)

win_size = FEATURE_WINDOW_SIZE 
block_size = (32, 32)
block_stride = block_size
cell_size = (16, 16)
num_bins = 9

HOG = cv2.HOGDescriptor(win_size, block_size, block_stride, cell_size, num_bins)

def extract_features(Image):

    resized_image = cv2.resize(Image, FEATURE_WINDOW_SIZE)
        
    feature = HOG.compute(resized_image).flatten()

    # feature = local_binary_pattern(resized_image, lbp_n_points, lbp_radius, method='uniform').flatten()

    # i_image = integral_image(resized_image)
    # feature = haar_like_feature(i_image, 0, 0, FEATURE_WINDOW_SIZE[0], FEATURE_WINDOW_SIZE[1])

    if feature.max() > 0:
        feature /= feature.max()

    return feature


for index, image_properties in positive_samples_df.iterrows():

    image_path = os.path.join("../../datasets/faces/images", image_properties["image_name"])
    gray_image = cv2.imread(image_path, 0)

    x0 = int(image_properties["x0"])
    x1 = int(image_properties["x1"])
    y0 = int(image_properties["y0"])
    y1 = int(image_properties["y1"])

    gray_face = gray_image[y0:y1 ,x0:x1]
    feature_vector.append(extract_features(gray_face))
    label_vector.append(1)


for image_path in positive_samples_paths:

    gray_face = cv2.imread(image_path, 0)
    feature_vector.append(extract_features(gray_face))
    label_vector.append(1)


for image_path in negative_samples_paths:

    gray_not_face = cv2.imread(image_path, 0)
    feature_vector.append(extract_features(gray_not_face))
    label_vector.append(0)


# 3. Train and Save the model

In [12]:
x_train, x_test, y_train, y_test = train_test_split(feature_vector, label_vector, test_size=0.05, random_state=42)

face_classifer = LinearSVC().fit(x_train, y_train)

score = face_classifer.score(x_test, y_test)

print("Accuracy: {:.2f}".format(score))

joblib.dump(face_classifer, "FaceClassifier{}({:.2f}).pkl".format(FEATURE_WINDOW_SIZE ,score))

c:\Tools\anaconda3\Lib\site-packages\sklearn\svm\_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(


Accuracy: 0.92


c:\Tools\anaconda3\Lib\site-packages\sklearn\svm\_base.py:1242: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


['FaceClassifier(64, 64)(0.92).pkl']

In [49]:
def segment_skin(image):
  """
  Segments skin pixels based on chrominance analysis.

  Args:
    image: A BGR image.

  Returns:
    skin_mask: A binary mask (0 or 255) where 255 represents skin pixels.
  """
  # Convert to YCbCr color space
  ycrcb = cv2.cvtColor(image, cv2.COLOR_BGR2YCrCb)

  # Define chrominance thresholds for skin
  lower_thresh = (0, 133, 77)
  upper_thresh = (255, 173, 127)

  # Create binary mask based on thresholds
  skin_mask = cv2.inRange(ycrcb, lower_thresh, upper_thresh)

  # Apply morphological operations for smoothing
  kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (3, 3))
  skin_mask = cv2.morphologyEx(skin_mask, cv2.MORPH_OPEN, kernel, iterations=2)
  skin_mask = cv2.morphologyEx(skin_mask, cv2.MORPH_CLOSE, kernel, iterations=2)

  return skin_mask

def union_boxes_min_max(boxes):
  """
  Unifies a list of bounding boxes using minimum and maximum values.

  Args:
    boxes: A list of bounding boxes represented as (x_min, y_min, x_max, y_max) tuples.

  Returns:
    unified_box: A tuple representing the unified bounding box (x_min, y_min, x_max, y_max).
  """
  min_x, min_y, max_x, max_y = 680, 680, 0, 0

  for box in boxes:
    x_min, y_min, x_max, y_max = box
    min_x = min(min_x, x_min)
    min_y = min(min_y, y_min)
    max_x = max(max_x, x_max)
    max_y = max(max_y, y_max)

  unified_box = (min_x, min_y, max_x, max_y)
  return unified_box



def filter_boxes_with_skin_mask(boxes, skin_mask):
  """
  Filters bounding boxes based on overlap with a skin mask.

  Args:
    boxes: A list of bounding boxes (x_min, y_min, x_max, y_max)
    skin_mask: A binary mask (0 or 255) where 255 represents skin pixels.

  Returns:
    filtered_boxes: A list of remaining bounding boxes after filtering.
  """
  filtered_boxes = []
  for box in boxes:
    x_min, y_min, x_max, y_max = box
    # Calculate area of the bounding box
    box_area = (x_max - x_min) * (y_max - y_min)
    # Get the portion of the bounding box within the skin mask
    intersection_mask = skin_mask[y_min:y_max, x_min:x_max]
    intersection_area = cv2.countNonZero(intersection_mask)
    # Calculate intersection-over-union (IoU) ratio
    iou = intersection_area / box_area

    # Define threshold for acceptable overlap with skin mask
    threshold = 0.3

    if iou >= threshold:
      filtered_boxes.append(box)

  return filtered_boxes  

# 4. Real-Time Testing

In [55]:
camera = cv2.VideoCapture(0)

window_sizes = [(128, 128)]

while camera.isOpened():

    detected_faces = []

    _, frame = camera.read()
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    skin_mask = segment_skin(frame)
    
    for window_size in window_sizes:

        y_stride = window_size[0] // 8
        x_stride = window_size[1] // 8

        for y in range(0, gray_frame.shape[0] - window_size[0], y_stride):
            for x in range(0, gray_frame.shape[1] - window_size[1], x_stride):
                # Extract window and predict
                window = gray_frame[y : y+window_size[0], x : x+window_size[1]]

                [prediction] = face_classifer.predict([extract_features(window)])

                # Check prediction and register face
                if prediction == 1:  # Adjust threshold based on your model
                    detected_faces.append([x, y, x + window_size[1], y + window_size[0]])


    filtered_faces = filter_boxes_with_skin_mask(detected_faces, skin_mask)

    filtered_face = union_boxes_min_max(filtered_faces)

    cv2.rectangle(frame, (filtered_face[0], filtered_face[1]), (filtered_face[2], filtered_face[3]), (0, 255, 0), 2)

    # Display the frame with the detection result
    cv2.imshow('Face Detection', frame)
    # Press 'q' to exit the loop
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the video capture object and close the OpenCV windows
camera.release()
cv2.destroyAllWindows()
